# **Tarea 1** (fecha de entrega: 17 de Junio 2023)
## Estudiante: Enzo Loiza B. (e.loi@ing.puc.cl)

Esta tarea consiste en 7 preguntas, de las cuales solo se considerarán las 6 mejores para el cálculo de la nota. **Se espera el codigo asociado a la respuesta y comentarios que hagan comprensible la lectura de la solución**. Puede entregar la tarea en el formato que estime conveniente (doc, pdf, script de R, url de Kaggle) pero tenga en cuenta que debe adjuntar el código utilizado. Se recomienda el uso de un notebook de R en la plataforma Kaggle (recordar dejar el notebook compartido con el ayudante para que este pueda abrirlo).



# Enunciado

Tu empresa fue contratada por la Asociación de Navegación Marítima de Chile para manejar la información de buques, navieras, y distintas rutas que bueques realizan para las navieras. Similarmente, se guardará la información de personal marino, y en cuales buques estos trabajan. Por ahora, tu desarrollo te ha llevado definir el siguiente esquema:

* $\color{green}{\textbf{Buques}}(\color{red}{\underline{\text{buqueId}}}, \color{red}{\text{nombre}}, \color{red}{\text{capacidad}}, \color{red}{\text{capitanId}})$ 
* $\color{green}{\textbf{Navieras}}(\color{red}{\underline{\text{navieraId}}}, \color{red}{\text{nombre}}, \color{red}{\text{pais}})$
* $\color{green}{\textbf{Navegaciones}}(\color{red}{\underline{\text{buqueId}}}, \color{red}{\underline{\text{navieraId}}},  \color{red}{{\text{ruta}}})$
* $\color{green}{\textbf{Personal}}(\color{red}{\underline{\text{personalId}}}, 
\color{red}{{\text{nombre}}}, \color{red}{{\text{edad}}},  \color{red}{\text{nacionalidad}})$
* $\color{green}{\textbf{TrabajaEn}}(\color{red}{\underline{\text{personalId}}}, 
\color{red}{\underline{\text{buqueId}}})$

Cómo un ejemplo, más abajo se muestra una instancia válida de tu esquema (puede ser que se revise la tarea con otra instancia). Es importante entender que un mismo buque puede ser contratado por varias naveras para realizar distintas rutas de trasbordo. Similarmente, una persona puede trabajar en más de un buque.


## Reglas: 
 - Las tablas que se entreguen como respuestas **no deben contener tuplas ni atributos adicionales a lo que se pregunta**.
 - No deben permitir que R modifique el nombre de un atributo por su cuenta (al hacer un inner_join de una tabla con ella misma por ejemplo), para evitar esto usen el comando rename.
 - Solo se debe responder usando los operadores vistos durante la segunda parte de la clase 2:
  - select
  - filter
  - union
  - rename
  - crossing
  - join
  - setdiff
  - anti_join

# Bases de datos

In [2]:
library(tidyverse)

In [3]:
buques <- tribble(
  ~buqueId, ~nombre, ~capacidad, ~capitanId,
  #-------/---------/----------/-----------
  "b1", "El Inundado", 4000, "p22",
  "b2", "Gotas Saladas", 2010, "p1",
  "b3", "Falucho", 4085, "p21",
  "b4", "Lento", 2989, "p33"
)

navieras <- tribble(
  ~navieraId, ~nombre, ~pais,
  #-------/---------/----------
  "n1", "Lagos del Sur", "Chile",
  "n2", "Somarco", "Chile",
  "n3", "Ultramar", "Argentina",
  "n4", "Broom", "Chile"
)

navegaciones <- tribble(
  ~buqueId, ~navieraId, ~ruta,
  #-------/---------/----------
  "b1", "n1", "Chiloe Norte",
  "b2", "n1", "Cruce Lago General Carrera",
  "b1", "n2", "Puerto Murta - Chile Chico",
  "b3", "n3", "Chaiten - Hornopiren"
)

personal <- tribble(
  ~personalId, ~nombre, ~edad, ~nacionalidad,
  #-------/---------/----------/-----------
  "p1", "Ian McIntosh", 32, "Escocia",
  "p2", "Christian Rutter", 24, "Alemania",
  "p3", "Mark Boyle", 33, "EE.UU.",
  "p4", "Victoria Kostylev", 48, "Chile",
  "p5", "Ian Stolberg", 44, "Suecia"
)

trabajaEn <- tribble(
  ~personalId, ~buqueId,
  #-------/---------
  "p1", "b2",
  "p1", "b4",
  "p3", "b4",
  "p4", "b4",
  "p2", "b3",
  "p2", "b2",
  "p3", "b3"
)

In [4]:
buques

buqueId,nombre,capacidad,capitanId
<chr>,<chr>,<dbl>,<chr>
b1,El Inundado,4000,p22
b2,Gotas Saladas,2010,p1
b3,Falucho,4085,p21
b4,Lento,2989,p33


In [5]:
navieras

navieraId,nombre,pais
<chr>,<chr>,<chr>
n1,Lagos del Sur,Chile
n2,Somarco,Chile
n3,Ultramar,Argentina
n4,Broom,Chile


In [6]:
navegaciones

buqueId,navieraId,ruta
<chr>,<chr>,<chr>
b1,n1,Chiloe Norte
b2,n1,Cruce Lago General Carrera
b1,n2,Puerto Murta - Chile Chico
b3,n3,Chaiten - Hornopiren


In [7]:
personal

personalId,nombre,edad,nacionalidad
<chr>,<chr>,<dbl>,<chr>
p1,Ian McIntosh,32,Escocia
p2,Christian Rutter,24,Alemania
p3,Mark Boyle,33,EE.UU.
p4,Victoria Kostylev,48,Chile
p5,Ian Stolberg,44,Suecia


In [8]:
trabajaEn

personalId,buqueId
<chr>,<chr>
p1,b2
p1,b4
p3,b4
p4,b4
p2,b3
p2,b2
p3,b3


# Pregunta 1
Encuentre los nombres de todos los buques con capacidad mayor o igual a 4000.

## Respuesta

In [43]:
filter(buques, 4000<=capacidad) %>% # filtramos por capacidad mayor o igual a 4000,
    select("nombre")                # seleccionamos sólo el nombre

nombre
<chr>
El Inundado
Falucho


# Pregunta 2
Encuentre los identificadores (personalId) de todo el personal que trabaja en el buque "Falucho" o "Gotas Saladas".

## Respuesta

In [44]:
table <- left_join(trabajaEn, buques, by = "buqueId") %>%
    filter(nombre == "Falucho" | nombre == "Gotas Saladas")       # hacemos una tabla uniendo por la izquierda las asignaciones de trabajo y los buques elegidos
semi_join(personal, table, by = "personalId") %>%                 # hacemos un semi join (intersección) de personal con la tabla recien creada.
    select(personalId)                                            # seleccionamos sólo la columna de id de personal

personalId
<chr>
p1
p2
p3


# Pregunta 3
Para validar si sus datos son consistente, asegure que cada capitan de un bote también está registrado en la tabla $\texttt{Personal}$. Para esto, escriba la operación que encuentre todos los id de los capitanes que **no** aparecen en la tabla de personal. 


### Hint de como usar setdiff y anti_join

* **setdiff**: diferencia de conjuntos, si tienen una tabla y quieren restar las observaciones de otra pueden usar este comando. Ejemplo, a la tabla "personal" quiero quitarle las dos primeras filas (personal[1:2,]).



In [45]:
personal %>% setdiff(personal[1:2,])

personalId,nombre,edad,nacionalidad
<chr>,<chr>,<dbl>,<chr>
p3,Mark Boyle,33,EE.UU.
p4,Victoria Kostylev,48,Chile
p5,Ian Stolberg,44,Suecia


* **anti_join**: el anti_join funciona de forma parecida al setdiff, sin embargo en vez de restar las observaciones repetidas, elimina segun un conjunto de atributos repetido. Por ejemplo si a la tabla "personal" queremos quitarle las filas cuyo **id** que se encuentran en la tabla "trabaja_en"

In [46]:
personal %>% anti_join(trabajaEn, by="personalId")

personalId,nombre,edad,nacionalidad
<chr>,<chr>,<dbl>,<chr>
p5,Ian Stolberg,44,Suecia


## Respuesta

In [47]:
buques %>% rename("personalId" = "capitanId") %>% # renombramos la columna capitanId por personalId para que sea comparable con la df de personal,
    anti_join(personal, by = "personalId") %>%    # hacemos un anti join para elegir sólo los id que no están en la df de personal,
    select(personalId)                            # seleccionamos solo la columna personalId

personalId
<chr>
p22
p21
p33


# Pregunta 4
Encuentre los **nombres** de el personal, con el nombre de los buques en los que trabajan, su edad y su nacionalidad, para los buques con capacidad menor o igual a 4000.

## Respuesta

In [48]:
personal %>% left_join(trabajaEn, by = "personalId") %>%   # unimos por la izquierda (para envitar sobredatos) las tablas personal y trabajaEn
    left_join(buques, by = "buqueId") %>%                  # unimos por la izquierda (para envitar sobredatos) la tabla actual y buques
    filter(capacidad<=4000) %>%                            # filtramos para buques con capacidad igual o menor que 4000,
    select(c("nombre.x","nombre.y","edad","nacionalidad")) # seleccionamos las columnas que se nos preguntan.

nombre.x,nombre.y,edad,nacionalidad
<chr>,<chr>,<dbl>,<chr>
Ian McIntosh,Gotas Saladas,32,Escocia
Ian McIntosh,Lento,32,Escocia
Christian Rutter,Gotas Saladas,24,Alemania
Mark Boyle,Lento,33,EE.UU.
Victoria Kostylev,Lento,48,Chile


# Pregunta 5
Encuentre los nombres de las personas trabajando para más de un buque.



### Hint
Puede hacer un join de una tabla consigo misma por algún atributo en común, y luego filtrar los resultados con el operador "<" otros dos atributos que se desea que sean distintos. Ejemplo:

In [49]:
navegaciones %>% inner_join(navegaciones, by='navieraId') %>% filter(buqueId.x < buqueId.y)

Warning message in inner_join(., navegaciones, by = "navieraId"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 1 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


buqueId.x,navieraId,ruta.x,buqueId.y,ruta.y
<chr>,<chr>,<chr>,<chr>,<chr>
b1,n1,Chiloe Norte,b2,Cruce Lago General Carrera


**Ojo, en este caso como ambas tablas tienen las mismas columnas, R les pone un sufijo a las columnas para diferenciarlas. Es mejor cambiarles el nombre previamente.**

## Respuesta

In [50]:
trabajaEn  %>% inner_join(personal, by = "personalId") %>% # hacemos un inner join para linkear los id con los nombres del personal
    inner_join(trabajaEn, by = "personalId") %>%           # usamos el hint de arriba: auto inner join
    filter(buqueId.x < buqueId.y) %>%                      # filtramos para que sean distintos los valores de buques
    select(nombre)                                           # retornamos sólo los nombres.
    

Warning message in inner_join(., trabajaEn, by = "personalId"):
“Detected an unexpected many-to-many relationship between `x` and `y`.
ℹ Row 1 of `x` matches multiple rows in `y`.
ℹ Row 1 of `y` matches multiple rows in `x`.
ℹ If a many-to-many relationship is expected, set `relationship =
  "many-to-many"` to silence this warning.”


nombre
<chr>
Ian McIntosh
Christian Rutter
Mark Boyle


# Pregunta 6
Encuentre el nombre de la persona de mayor edad de la lista de personal.

**Observacion**: NO se puede usar la funcion max()


### Hint
Primero encuentre todas las personas que son menores a alguien. Luego use ese resultado para calcular los que **siempre** son mayores. Intenta usar crossing

In [51]:
personal_1 <- personal
personal_2 <- personal

names(personal_1) <- c("personalId_1","nombre_1","edad_1","nacionalidad_1")
names(personal_2) <- c("personalId_2","nombre_2","edad_2","nacionalidad_2")

#crossing tiene problemas con los nombres de atributos repetidos, por lo tanto los cambiamos previamente
personal_1 %>% crossing (personal_2)
# Pregunta: ¿Si es que comparo las edades de todos, que es lo que cumple la persona de mayor edad?

personalId_1,nombre_1,edad_1,nacionalidad_1,personalId_2,nombre_2,edad_2,nacionalidad_2
<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<chr>
p1,Ian McIntosh,32,Escocia,p1,Ian McIntosh,32,Escocia
p1,Ian McIntosh,32,Escocia,p2,Christian Rutter,24,Alemania
p1,Ian McIntosh,32,Escocia,p3,Mark Boyle,33,EE.UU.
p1,Ian McIntosh,32,Escocia,p4,Victoria Kostylev,48,Chile
p1,Ian McIntosh,32,Escocia,p5,Ian Stolberg,44,Suecia
p2,Christian Rutter,24,Alemania,p1,Ian McIntosh,32,Escocia
p2,Christian Rutter,24,Alemania,p2,Christian Rutter,24,Alemania
p2,Christian Rutter,24,Alemania,p3,Mark Boyle,33,EE.UU.
p2,Christian Rutter,24,Alemania,p4,Victoria Kostylev,48,Chile


## Respuesta

In [52]:
personal_1 <- personal
personal_2 <- personal

names(personal_1) <- c("personalId_1","nombre_1","edad_1","nacionalidad_1")
names(personal_2) <- c("personalId_2","nombre_2","edad_2","nacionalidad_2")

# Hasta acá es el mismo proceso que el hint (creamos dos df con distintos nombres de columnas para cruzar)

tabla <- personal_1 %>% crossing (personal_2) %>%  # creamos una tabla con crossing
    filter(edad_1 < edad_2)                        # filtramos comparando que la edad_1 sea menor que edad_2. La persona de mayor edad no aparecerá en la columna edad_1
personal_1 %>%                                     # por lo que a la df de personal_1
    anti_join(tabla, by = "nombre_1") %>%          # hacemos un anti join usando la columna edad_1, la única que aparecerá es la persona de mayor edad.
    select(nombre_1)                               # mostramos sólo el nombre de esta persona.

nombre_1
<chr>
Victoria Kostylev


# Pregunta 7
Encuentre los nombres de todos los buques trabajando para una naviera chilena.

## Respuesta

In [53]:
navegaciones %>% inner_join(navieras, by = "navieraId") %>%    # hacemos un inner join para juntar las df de navegaciones y navieras
    inner_join(buques, by = "buqueId") %>%                     # otro join para obtener los nombres de los buques
    filter(pais == "Chile") %>%                                # filtramos las navieras por país: Chile 
    select(nombre.y)                                           # seleccionamos sólo la columna de nombre, nombre.y es el de buques.

nombre.y
<chr>
El Inundado
Gotas Saladas
El Inundado
